In [ ]:
import tensorflow as tf
tf.random.set_seed(42)
import numpy as np
np.random.seed(42)
import time
import pickle

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
import plotly.offline as pyo
from plotly import subplots
import plotly.graph_objects as go
import statsmodels.api as sm
import statsmodels.tsa.api as smt

from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Model
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import LSTM, Dense,Input,concatenate
from tensorflow.keras.layers import *
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
import lightgbm as lgb

from datetime import datetime, timedelta


from baseFunctions import *
from data_helpers import processData6, featureEngineering, getSequencesFast, removeOutliers


In [ ]:
data, propDicts, flippedPropDicts = processData6()
data, timeFeatures = featureEngineering(data)

In [ ]:
trainF = [
       'store_nbr', 'family', 
       #'sales', 
       'onpromotion',# 'dataT',
       'city', 'state', 'type', 'cluster', 'dcoilwtico', 'holidayType',
       'description', 'transferred', 
       #'transactions', 
       'store_closed',
       'linear_time', 'day_of_year', 'day_of_year_f12_0', 'day_of_year_f104_0',
       'day_of_year_f24_0', 'day_of_year_f52_0', 'weekday', 'month'
       ]

trainF2 = trainF + ['sales']



n_predictedValues = 16
look_back = 100
zScoreNorm = False
logTransform = True

# Date string
date_string = "2017-05-01"
date_object = datetime.strptime(date_string, '%Y-%m-%d')
days_ago = date_object - timedelta(days=(look_back + n_predictedValues -1 +2))
days_ago_string = days_ago.strftime('%Y-%m-%d')
days_ago_string

maskTrain = data.date < date_string
maskTest = data.date  > days_ago_string #"2017-03-03" #42days + 15 day between (15 because we only want to iterate one value more from the test set)

log = {}

train = data.loc[data.dataT == 'train']

    

    
stdDict = {}
meanDict= {}
X_train = []
X_test  = []
y_train = []
y_test  = []
std_train, std_test = [], []
mean_train, mean_test = [], []
init= False
for familyId in [0]: #data.family.unique():
       # start with only some families
       if familyId > 8:
          continue

       print(familyId)
       familyDf = train.loc[train.family==familyId]  

       for storeId in data.store_nbr.unique():
              storeDf = familyDf.loc[(familyDf.store_nbr == storeId)] 

              # ln tranformation
              if logTransform:
                     storeDf.loc[:,'sales'] = np.log(storeDf.sales + 1)
              if zScoreNorm:
                     X_train0,y_train0,mean,std = getSequencesFast(storeDf.loc[maskTrain], trainF, look_back, n_predictedValues, zScoreNorm=zScoreNorm)
                     X_test0, y_test0           = getSequencesFast(storeDf.loc[maskTest], trainF, look_back, n_predictedValues, zScoreNorm=False, applyZScoreNorm=True, meanZ=mean, stdZ=std)
                     stdDict[storeId] = std
                     meanDict[storeId] = mean
              else:
                     X_train0,y_train0 = getSequencesFast(storeDf.loc[maskTrain], trainF, look_back, n_predictedValues, zScoreNorm=False)
                     X_test0, y_test0  = getSequencesFast(storeDf.loc[maskTest], trainF, look_back, n_predictedValues, zScoreNorm=False, applyZScoreNorm=False)  
              
              if init:
                     X_train = np.concatenate((X_train, X_train0), axis=0)
                     X_test  = np.concatenate((X_test,  X_test0), axis=0)
                     y_train = np.concatenate((y_train, y_train0), axis=0)
                     y_test  = np.concatenate((y_test,  y_test0), axis=0)
                     if zScoreNorm:
                            std_train = np.concatenate((std_train, np.ones(y_train0.shape)*std), axis=0)
                            std_test  = np.concatenate((std_test,  np.ones(y_test0.shape)*std), axis=0)
                            mean_train = np.concatenate((mean_train, np.ones(y_train0.shape)*mean), axis=0)
                            mean_test = np.concatenate((mean_test, np.ones(y_test0.shape)*mean), axis=0)
              else:
                     X_train, X_test, y_train,y_test = X_train0, X_test0, y_train0, y_test0
                     if zScoreNorm:
                            std_train  =np.ones(y_train0.shape)*std
                            std_test   =np.ones(y_test0.shape)*std
                            mean_train =np.ones(y_train0.shape)*mean
                            mean_test  =np.ones(y_test0.shape)*mean
                     init=True 


 

In [ ]:
y_train, y_test

In [ ]:
np.savez('train_test_Sequences_100_16_StdNormalized_fam0-7.npz', arr1=X_train, arr2=y_train, arr3=std_train, arr4=X_test, arr5=y_test, arr6=std_test)

# LGBM

using a ln transformation and rmse vs. rmsle: -> maybe a bit less overfitting (0.628/0.561 vs. 0.581/0.588)

In [ ]:
# Set parameters for LGBM model
params = {
    'objective': 'regression',  # Assuming you're doing regression
    'metric': 'rmsle',  # Mean squared error
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1
}   

# Train the model
num_round = 10  
gbms = [lgb.train(params, lgb.Dataset(X_train, label=y_train[:, i]),num_round, valid_sets=[lgb.Dataset(X_test, label=y_test[:,i])]) for i in range(y_train.shape[1])]   


forecast = np.column_stack([gbm.predict(X_train, num_iteration=gbm.best_iteration) for gbm in gbms])
if (forecast<0).any():
    print('negative values!!!')
    forecast = np.clip(forecast, 0, 1e29)
if zScoreNorm:
    forecast = forecast *std_train  + mean_train
    y_train = y_train *std_train + mean_train
rmsleTrain = np.sqrt(mean_squared_log_error(forecast, y_train))
forecast = np.column_stack([gbm.predict(X_test, num_iteration=gbm.best_iteration) for gbm in gbms])
if zScoreNorm:
    forecast = forecast *std_test  + mean_test
    y_test = y_test*std_test + mean_test
rmsleTest = np.sqrt(mean_squared_log_error(forecast, y_test))
print('familyId:', familyId, 'errors:  ', round(rmsleTrain,3), round(rmsleTest,3), y_train.shape[0], y_test.shape[0])


In [ ]:
#LOG version
params = {
    'objective': 'regression',  # Assuming you're doing regression
    'metric': 'rmse',#'rmsle',  # Mean squared error
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1
}   

# Train the model
num_round = 100  
gbms = [lgb.train(params, lgb.Dataset(X_train, label=y_train[:, i]),num_round, verbose_eval=False, valid_sets=[lgb.Dataset(X_test, label=y_test[:,i])]) for i in range(y_train.shape[1])]   


forecast = np.column_stack([gbm.predict(X_train, num_iteration=gbm.best_iteration) for gbm in gbms])
if logTransform:
    forecast0 = np.exp(forecast) - 1
    y_train0  = np.exp(y_train) - 1
    rmsleTrain = np.sqrt(mean_squared_log_error(forecast0, y_train0))
forecast = np.column_stack([gbm.predict(X_test, num_iteration=gbm.best_iteration) for gbm in gbms])
if logTransform:
    forecast1 = np.exp(forecast) - 1
    y_test1   = np.exp(y_test) - 1
    rmsleTest = np.sqrt(mean_squared_log_error(forecast1, y_test1))
print('familyId:', familyId, 'errors:  ', round(rmsleTrain,3), round(rmsleTest,3))


In [ ]:
#familyId: 0 errors:   0.695 0.634

# facebook prophet

In [ ]:
from fbprophet import Prophet

In [ ]:
# Assuming df is a pandas dataframe with 'ds' and 'y' columns
m = Prophet()
m.fit(df)

# Make future dataframe for 16 periods
future = m.make_future_dataframe(periods=16)
forecast = m.predict(future)


# Transformer

idea: we could use positional embeddings to define the relation between store and product

In [ ]:
# create training data for transformer / nn architectures
from data_helpers import create_sequences
def getSequencesFastNOTFlattened(train, trainF, look_back, n_predictedValues, zScoreNorm = False, logTransf=False, applyZScoreNorm = False, meanZ = 0, stdZ = 0,):
    # zscore over all values -> not ideal bc test data
    if zScoreNorm:
        #mean = train.sales.mean()
        mean = 0 # modified zScore, not in mean = 0
        std = max(train.sales.std(), 1)
        train.loc[:,'sales'] = (train.sales - mean) / std
    if applyZScoreNorm:
        train.loc[:,'sales'] = (train.sales-meanZ)/stdZ
    if logTransf:
        train.loc[:,'sales'] = np.log(train.sales + 1)

    trainF2 = trainF + ['sales']


    pastS   = create_sequences(train[trainF2].to_numpy(), look_back)                                   #past sequence
    futureS = create_sequences(train[trainF].iloc[look_back:-1].to_numpy(), n_predictedValues)        #future sequence
    label   = create_sequences(train[['sales']].iloc[look_back:-1].to_numpy(), n_predictedValues) #label

    maxLen = train.shape[0] - look_back-n_predictedValues
    pastS = pastS[0:maxLen,:,:].transpose([0,2,1])
    futureS = futureS[0:maxLen,:,:].transpose([0,2,1])
    label = label[0:maxLen,:].transpose([0,2,1])

    if zScoreNorm:
        return pastS, futureS, label, mean, std
    elif applyZScoreNorm:
        return pastS, futureS, label
    else:
        return pastS, futureS, label

trainF = [
       'store_nbr', 'family', 
       #'sales', 
       'onpromotion',# 'dataT',
       'city', 'state', 'type', 'cluster', 'dcoilwtico', 'holidayType',
       'description', 'transferred', 
       #'transactions', 
       'store_closed',
       'linear_time', 'day_of_year', 'day_of_year_f12_0', 'day_of_year_f104_0',
       'day_of_year_f24_0', 'day_of_year_f52_0', 'weekday', 'month'
       ]

trainF2 = trainF + ['sales']



n_predictedValues = 16
look_back = 100
zScoreNorm = False
logTransform = True

# Date string
date_string = "2017-05-01"
date_object = datetime.strptime(date_string, '%Y-%m-%d')
days_ago = date_object - timedelta(days=(look_back + n_predictedValues -1 +2))
days_ago_string = days_ago.strftime('%Y-%m-%d')
days_ago_string

maskTrain = data.date < date_string
maskTest = data.date  > days_ago_string #"2017-03-03" #42days + 15 day between (15 because we only want to iterate one value more from the test set)

log = {}

train = data.loc[data.dataT == 'train']

    

    
stdDict = {}
meanDict= {}
X_trainS0 = []
X_testS0  = []
X_trainS1 = []
X_testS1  = []
y_train = []
y_test  = []
std_train, std_test = [], []
mean_train, mean_test = [], []
init= False
for familyId in [0]: #data.family.unique():
       print(familyId)
       familyDf = train.loc[train.family==familyId]  

       for storeId in data.store_nbr.unique():
              if storeId >1:
                continue
              print(storeId)
              storeDf = familyDf.loc[(familyDf.store_nbr == storeId)] 
              if zScoreNorm:
                X_train0,X_train1,y_train0,mean,std = getSequencesFastNOTFlattened(storeDf.loc[maskTrain], trainF, look_back, n_predictedValues, zScoreNorm=zScoreNorm)
                X_test0,X_test1, y_test0            = getSequencesFastNOTFlattened(storeDf.loc[maskTest], trainF, look_back, n_predictedValues, zScoreNorm=False, applyZScoreNorm=True, meanZ=mean, stdZ=std)       
                stdDict[storeId] = std
                meanDict[storeId] = mean
              else:
                X_train0,X_train1,y_train0 = getSequencesFastNOTFlattened(storeDf.loc[maskTrain], trainF, look_back, n_predictedValues, zScoreNorm=False, logTransf=logTransform)
                X_test0,X_test1, y_test0   = getSequencesFastNOTFlattened(storeDf.loc[maskTest], trainF, look_back, n_predictedValues, zScoreNorm=False, logTransf=logTransform, applyZScoreNorm=False, meanZ=0, stdZ=0) 
              if init:
                     X_trainS0 = np.concatenate((X_trainS0, X_train0), axis=0)
                     X_testS0  = np.concatenate((X_testS0,  X_test0), axis=0)
                     X_trainS1 = np.concatenate((X_trainS1, X_train1), axis=0)
                     X_testS1  = np.concatenate((X_testS1,  X_test1), axis=0)
                     y_train = np.concatenate((y_train, y_train0), axis=0)
                     y_test  = np.concatenate((y_test,  y_test0), axis=0)
                     if  zScoreNorm:
                        std_train = np.concatenate((std_train, np.ones(y_train0.shape)*std), axis=0)
                        std_test  = np.concatenate((std_test,  np.ones(y_test0.shape)*std), axis=0)
                        mean_train = np.concatenate((mean_train, np.ones(y_train0.shape)*mean), axis=0)
                        mean_test = np.concatenate((mean_test, np.ones(y_test0.shape)*mean), axis=0)
              else:
                     X_trainS0, X_trainS1 = X_train0, X_train1
                     X_testS0, X_testS1   = X_test0, X_test1
                     y_train,y_test = y_train0, y_test0
                     if zScoreNorm:
                        std_train  =np.ones(y_train0.shape)*std
                        std_test   =np.ones(y_test0.shape)*std
                        mean_train =np.ones(y_train0.shape)*mean
                        mean_test  =np.ones(y_test0.shape)*mean
                     init=True 

X_test = [X_testS0, X_testS1]
X_train = [X_trainS0, X_trainS1]
X_train[0].shape
 

In [ ]:
X_trainS0.shape, X_testS0.shape, y_train.shape, y_test.shape

## transformer from scratch

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from Transformer import encoder, decoder, embedding, attention, feed_forward
from Transformer import transformer

In [ ]:
#sanity checks
embed_pt = PositionalEmbedding(sequence_length=100, d_model=21)

s0 = embed_pt(X_trainS0[0:10,:,:])

embed_pt = PositionalEmbedding(sequence_length=16, d_model=20)

s1 = embed_pt(X_trainS1[0:10,:,:])
print('embedding',s1.shape)

sample_ca = CrossAttention(num_heads=2, key_dim=512)

print('crossAttnetion',sample_ca(x=s1,context=s0).shape)

# Instantiate the encoder.
sample_encoder = Encoder(num_layers=4,
                         d_model=21,
                         num_heads=8,
                         dff=64,
                         sequence_length=100)

sample_encoder_output = sample_encoder(X_trainS0[0:10,:,:], training=False)

print('encoder',sample_encoder_output.shape)  # Shape `(batch_size, input_seq_len, d_model)`.

# Instantiate the decoder.
sample_decoder = Decoder(num_layers=4,
                         d_model=20,
                         num_heads=8,
                         dff=64,
                         sequence_length=16)

output = sample_decoder(
    x=s1,
    context=s0)

# Print the shapes.
print('decoder',output.shape)

transformer = Transformer(
    enc_dec_num_layers=1,
    input_seq_dim_enc = 21,#X_trainS0.shape[2],
    input_seq_dim_dec = 20,#X_trainS1.shape[2],
    num_heads=8,
    fully_connected_size=64,
    input_sequence_len=look_back,
    output_sequence_len=1, #n_predictedValues,
    dropout_rate=0)
    
output = transformer(([X_trainS0[0:10,:,:],X_trainS1[0:10,:,:]]))

print('transformer',output.shape)

In [ ]:
from keras import backend as K

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

In [ ]:
tf.keras.utils.set_random_seed(42)



transformer = Transformer(
    enc_dec_num_layers=1,
    input_seq_dim_enc = X_trainS0.shape[2],
    input_seq_dim_dec = X_trainS1.shape[2],
    num_heads=8,
    fully_connected_size=64,
    input_sequence_len=look_back,
    output_sequence_len=1, #n_predictedValues,
    dropout_rate=0)

transformer.compile(
    loss=root_mean_squared_error,  #if I log transform my data, the loss is just rmse
    optimizer='adam',
    metrics=['mae'])

transformer.fit(X_train, y_train,
    epochs=10,
    validation_data=(X_test, y_test), batch_size=64)



In [ ]:
forecast = transformer.predict(X_train)
if (forecast<0).any():
    print('negative values!!!')
    forecast = np.clip(forecast, 0, 1e29)
if zScoreNorm:
    forecast = forecast *std_train  + mean_train
    y_train = y_train *std_train + mean_train
rmsleTrain = np.sqrt(mean_squared_log_error(np.reshape(forecast,(-1,16)), np.reshape(y_train, (-1,16))))
forecast = transformer.predict(X_test)
if zScoreNorm:
    forecast = forecast *std_test  + mean_test
    y_test = y_test*std_test + mean_test
rmsleTest = np.sqrt(mean_squared_log_error(np.reshape(forecast,(-1,16)), np.reshape(y_test, (-1,16))))
print('errors:  ', round(rmsleTrain,3), round(rmsleTest,3))

# GRU

In [ ]:
from tensorflow.keras.layers import GRU

In [ ]:
tf.keras.utils.set_random_seed(42)

model = Sequential()
model.add(GRU(32, return_sequences=True, input_shape=(100, 20)))
model.add(GRU(32))
model.add(Dense(16))

model.compile(loss='rmse', optimizer='adam')
model.fit(X_train, y_train, epochs=10, batch_size=32,validation_data=(X_test, y_test))


In [ ]:
tf.keras.utils.set_random_seed(42)

# Define an input sequence.
encoder_inputs = Input(shape=(None, num_encoder_tokens))

# Create a GRU layer
encoder = GRU(latent_dim, return_state=True)

# Get the output and state
encoder_outputs, state_h = encoder(encoder_inputs)

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))

# Create another GRU layer
decoder_gru = GRU(latent_dim, return_sequences=True, return_state=True)

# Get the output and state
decoder_outputs, _ = decoder_gru(decoder_inputs, initial_state=state_h)

# Create a dense layer
decoder_dense = Dense(num_decoder_tokens, activation='softmax')

# Get the final output
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


# LSTM

In [ ]:
tf.keras.utils.set_random_seed(42)

# Define an input sequence.
encoder_inputs = Input(shape=(None, num_encoder_tokens))

# Create a LSTM layer
encoder = LSTM(latent_dim, return_state=True)

# Get the output and states
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))

# Create another LSTM layer
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)

# Get the output and states
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# Create a dense layer
decoder_dense = Dense(num_decoder_tokens, activation='softmax')

# Get the final output
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


# XGBoost